# Lightweight Fine-Tuning Project

In this notebook I am goint to apply parameter-efficient fine-tuning (PEFT) to boost the accuracy of detecting fake news from the [magnea/fake-news-formated](https://huggingface.co/datasets/magnea/fake-news-formated) dataset from HuggingFace🤗.

This dataset contains ~165k examples of news articles and a classification wether these news are legit or fake.

For fine-tuning I am going to apply **low-rank adaptation (LoRA)** to the **GPT-2** model. 

### Dataset and Preprocessing

I can see that the dataset is split about 90/10 into *training* and *test* data and that the news articles are seperated into title and content. 

In [4]:
from datasets import load_dataset

dataset = load_dataset("magnea/fake-news-formated")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'classification'],
        num_rows: 149049
    })
    test: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'classification'],
        num_rows: 16556
    })
})

For consistency I want to sort out all rows where only one of the parts is available. Additionally I am going to rename the *classification* column to *label* in order to comply with the huggingface Trainer API.

In [2]:
dataset= dataset.filter(lambda d: d['title'] != None and d['content'] != None)
dataset = dataset.rename_column('classification', 'label')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'label'],
        num_rows: 114522
    })
    test: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'label'],
        num_rows: 12728
    })
})

Next I am going to combine the *title* and *content* column into one in order to pass both in a standardized format into the model. Furthermore I am also mapping the labels to numeric values.

In [3]:
id2label={0: 'fake', 1: 'real'}
label2id={'fake': 0, 'real': 1}

def combine_fields(d):
	return {'combined': d['title'] + '\n\n' + d['content']}

def transform_label(d):
    d['label'] = label2id[d['label']]
    return d

splits = dataset.keys()
for split in splits:
    dataset[split] = dataset[split].map(combine_fields)
    dataset[split] = dataset[split].map(transform_label)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'label', 'combined'],
        num_rows: 114522
    })
    test: Dataset({
        features: ['id', 'dataset_id', 'title', 'content', 'label', 'combined'],
        num_rows: 12728
    })
})

In [4]:
dataset['train'][4711]

{'id': 'e52659cd236e805',
 'dataset_id': 1.0,
 'title': "South Africa's Dlamini-Zuma, ANC leadership contender, to become MP",
 'content': 'JOHANNESBURG (Reuters) - South African veteran politician and anti-apartheid activist Nkosazana Dlamini-Zuma, a leading contender to take over as head of the ruling ANC in December, will be sworn in as a member of parliament next week, a senior party official said on Friday. Dlamini-Zuma, the ex-wife of current ANC leader and South African President Jacob Zuma, does not hold a top position and could use a seat in parliament to raise her profile ahead of the party s December leadership conference.   She is going to be sworn in,  ANC Secretary General Gwede Mantashe was quoted as saying by the local EWN news network. The former health and foreign affairs minister s main opponent in the ANC leadership race is expected to be Deputy President Cyril Ramaphosa, a trade unionist-turned-business tycoon whom many investors would prefer to see running a count

Finally I am using the `AutoTokenizer` to tokenize the dataset for the GPT-2 model. I am using padding and truncation to allign all rows in order to allows batch processing.

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data(d):
    return tokenizer(d['combined'], padding='max_length', truncation=True)


tokenized = {}
for split in splits:
    tokenized[split] = dataset[split].map(preprocess_data, batched=True)

tokenized

{'train': Dataset({
     features: ['id', 'dataset_id', 'title', 'content', 'label', 'combined', 'input_ids', 'attention_mask'],
     num_rows: 114522
 }),
 'test': Dataset({
     features: ['id', 'dataset_id', 'title', 'content', 'label', 'combined', 'input_ids', 'attention_mask'],
     num_rows: 12728
 })}

In [6]:
tokenized['test']

Dataset({
    features: ['id', 'dataset_id', 'title', 'content', 'label', 'combined', 'input_ids', 'attention_mask'],
    num_rows: 12728
})

# Evaluationg the Foundation Model

I am using the `AutoModelForSequenceClassification` to load the GPT-2 model extended with a classification head for 2 classes and set `requires_grad=False` for all model parameters as we do not want to alter the model during evaluation. 

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'gpt2',
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
for param in model.parameters():
    param.requires_grad = False

model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

Finally I am using the `Trainer` class to evaluate the model performance on the *test* split of the dataset.

In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./benchmark",
        learning_rate=2e-3,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    ),
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.evaluate()

/tmp/ipykernel_23297/2265083618.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 1.5774004459381104,
 'eval_model_preparation_time': 0.0007,
 'eval_accuracy': 0.494343180389692,
 'eval_runtime': 175.1342,
 'eval_samples_per_second': 72.676,
 'eval_steps_per_second': 1.136}

The `'eval_accuracy': 0.494343180389692,` shows that the model is pretty much guessing the class resulting in a 50/50 chance to be right. So at the moment we can als flip a coin at much lower operational cost.

# Applying Low-Rank Adaptation (LoRA)

The schema below demonstrates how LoRA reduces the number of parameters that need to be adjusted.

![Schema of Low-Rank Adaptation(LoRA)](images/lora.png)

First I am creating a configuration for my LoRA application which I will explain below

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    bias='none',
    task_type='SEQ_CLS',
    target_modules=['c_attn', 'c_proj'],
    modules_to_save=['score']
)

`r` is the rank of the update matricies A and B and `lora_alpha` defines the scaling. These parameters define the scaling of the resulting adaptation matrix as 

![lora scaling formula](images/scaling.png)

with `r=4` and `lora_alpha=32` the scaling I picked is ``32/4 => 8``

`bias='none'` means that the biases of the model remain untouched

`task_type='SEQ_CLS'` tells the trainer that the models task is classification rather than generation

`target_modules=['c_attn', 'c_proj']` restricts the adaptation to the attention modules which should be enough and saves time

`modules_to_save=['score']` enables training of the classification head, if not set it remains the same

`lora_dropout=0.01` adds a little bit of dropout in order to prevent overfitting

The resulting model looks like this

In [9]:
lora_model = get_peft_model(model=model, peft_config=config)
lora_model.print_trainable_parameters()
lora_model

trainable params: 407,040 || all params: 124,848,384 || trainable%: 0.3260


/home/thorsten/code/nd608-generative-ai-project-1/.venv/lib/python3.12/site-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B)

### Running Fine-Tuning

In the following I am running 10 epochs of fine-tuning by applying LoRA. In order to save time I am only picking 128 random examples per epoch. Otherwise a full epoch would take about 55hrs on my RTX 5070ti (16GB).

In [11]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./lora-model",
        learning_rate=2e-3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        num_train_epochs=10,
        weight_decay=0.01,
    ),
    train_dataset=tokenized["train"].shuffle(seed=42).select(range(128)),
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

/tmp/ipykernel_77704/1966471401.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.687464,0.613294
2,No log,0.781059,0.625236
3,No log,0.682606,0.627121
4,No log,0.808708,0.668290
5,No log,0.823840,0.656662
6,No log,1.119632,0.664048
7,No log,1.151145,0.667112
8,No log,1.147137,0.677718
9,No log,1.319463,0.668133
10,No log,1.363969,0.673397


{'eval_loss': 0.6826058030128479,
 'eval_accuracy': 0.6271213073538655,
 'eval_runtime': 656.5363,
 'eval_samples_per_second': 19.387,
 'eval_steps_per_second': 1.212,
 'epoch': 10.0}

After training I am evaluation the fine-tuned model on the same `test` dataset split as for the foundation model benchmark. As a result we can see that the accuracy has improved by 13 points to about ~63% which is already better than guessing after only 2 hours of training on very few samples.

### Saving the model parameters

In [12]:
lora_model.save_pretrained("peft-fake-news", save_adapter=True, save_config=True)
lora_model.merge_and_unload()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

### Exploring results

For exploration I am going to pick 20 random rows from the `test` dataset split and compare the prediction with the actual classification.

In [18]:
import pandas as pd

examples = tokenized["test"].shuffle(seed=42).select(range(20))
df = pd.DataFrame(examples)
df = df[["combined", "label"]]
predictions = trainer.predict(examples)
df["predicted_label"] = np.argmax(predictions[0], axis=1)
df.head(20)

,combined,label,predicted_label
0,BREAKING: MASSIVE CYBER ATTACK ON ALL FEDERAL ...,1,1
1,Coronavirus May Mean Lights Out For Summer Cam...,1,0
2,Deadly Somalia blast reveals flaws in intellig...,0,0
3,Head of Justice National Security Division to ...,1,0
4,No link between hypertension drugs and COVID-1...,1,1
5,HERE’S THE LIST OF People We Elected Who Just ...,0,1
6,Brothers ID'd as suicide bombers in Belgium\n\...,1,1
7,"Puerto Rico evacuates area near crumbling dam,...",1,1
8,"Trump win, Democratic setbacks cloud Pelosi's ...",1,1
9,Trump Confidante Confirms: The Donald Doesn’t...,0,1


# Inference

I am going to test inference by loading the saved model and predicting a label for this [article from CBS News](https://www.cbsnews.com/news/white-house-ballroom-east-wing-demolition/).

In [6]:
title = "White House begins demolition of part of East Wing for Trump's ballroom"
content = """The Washington Post first reported on the demolition and published an image of the work. And on Monday, a pool reporter captured video of part of the East Wing being torn down. 

During an event Monday with the Louisiana State University baseball team at the White House, the president remarked on the construction, which he said "just started today."

"You know we're building — right behind us — we're building a ballroom," Mr. Trump said during the celebration of the 2025 NCAA champions in the White House East Room. He pointed out, "Right on the other side, you have a lot of construction going on, which you might hear periodically."

The president also referenced the construction during a meeting with GOP senators on the Rose Garden patio Tuesday. Mr. Trump overhauled the Rose Garden and covered the grass earlier this year to make it easier to host guests.
"""

cbs_news_article = title + '\n\n' + content

In [ ]:
from peft import AutoPeftModelForSequenceClassification
import torch

saved_model = AutoPeftModelForSequenceClassification.from_pretrained("peft-fake-news",  num_labels=2)

infer_input = tokenizer(cbs_news_article, padding="max_length", truncation=True, return_tensors='pt')
with torch.no_grad():
    infer_output = saved_model(**infer_input)
    logits = infer_output.logits

probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_class = torch.argmax(probabilities, dim=-1).numpy()[0]
print("Predicted class:", id2label[predicted_class])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]